In [3]:
pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests

# 데이터프레임 불러오기
df = pd.read_excel('선박 화재 발생 일자 데이터.xlsx', header=None)
df['year'] = df[0].dt.year
df['month'] = df[0].dt.month
df['day'] = df[0].dt.day
df = df.drop(0, axis=1)

apikey = "ikyOZ5uYRSGMjmebmJUhoQ"
stn_dic = {
    "부산광역시": 159,
    "전라남도": 168,
    "경상남도": 159,
    "강원특별자치도": 119,
    "경상북도": 143,
    "전북특별자치도": 146,
    "인천광역시": 112,
    "제주특별자치도": 184,
    "서울특별시": 108,
    "울산광역시": 152,
    "경기도": 119,
    "충청남도": 177
}

In [4]:
import requests
import time

def download_file(date, stn):
    url = f'https://apihub.kma.go.kr/api/typ01/url/kma_sfctm2.php?tm={date}&stn={stn}&help=0&authKey={apikey}'
    # print(url)
    for attempt in range(5):  # 5회 시도
        try:
            response = requests.get(url, timeout=10)  # 10초 타임아웃
            response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
            return response.content.decode('utf-8').splitlines()  # 성공적으로 데이터를 가져왔을 때
        except requests.exceptions.Timeout:
            print(f"Attempt {attempt + 1} timed out, retrying...")
            time.sleep(1)  # 1초 대기 후 재시도
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {date}, station {stn}: {e}")
            return None
    
    print(f"Failed to fetch data for {date}, station {stn} after multiple attempts.")
    return None


# 파일에서 읽어오기

In [20]:
def extract_weather_data(lines):
    split_data = lines.split()  # 공백 기준으로 split
    if split_data[0] == '#7777END':  # 데이터가 없는 경우
        return None
    else:
        return {
            'ws': split_data[3],    # 풍속(m/s)
            'pa': split_data[7],    # 기압(hPa)
            'ps': split_data[8],    # 해면 기압(hPa)
            'ta': split_data[11],   # 기온(C)
            'td': split_data[12],   # 이슬점온도(C)
            'hm': split_data[13],   # 상대습도(%)
            'rn': split_data[16],   # 일일 강수량(mm)
            'ss': split_data[33],   # 일조 (hr)
            'si': split_data[34],   # 일사 (MJ/m2)
            'ts': split_data[36],   # 지면온도 (C)
        }
    return None

# 파일 경로 지정
file_path = 'output_file충청남도.txt'  # 파일 경로를 지정하세요

# 파일 읽기
with open(file_path, 'r') as file:
    lines = file.readlines()  # 모든 줄을 리스트로 읽어옴

# 5번째 줄 가져오기 (인덱스는 0부터 시작하므로 4로 접근)
fifth_line = lines[4].strip()  # .strip()을 사용하여 줄바꿈 문자 제거

temp = extract_weather_data(fifth_line)

{'ws': '3.9',
 'pa': '1021.5',
 'ps': '1030.2',
 'ta': '4.2',
 'td': '-11.8',
 'hm': '30.0',
 'rn': '-9.0',
 'ss': '1.0',
 'si': '2.12',
 'ts': '17.8'}

# GET요청 후 바로 처리

In [5]:
def extract_weather_data(lines):
    if lines and len(lines) > 4:
        fourth_line = lines[4].strip()  # 5번째 줄 추출
        split_data = fourth_line.split()  # 공백 기준으로 split
        if split_data[0] == '#7777END':  # 데이터가 없는 경우
            return None
        else:
            return {
            'ws': split_data[3],    # 풍속(m/s)
            'pa': split_data[7],    # 기압(hPa)
            'ps': split_data[8],    # 해면 기압(hPa)
            'ta': split_data[11],   # 기온(C)
            'td': split_data[12],   # 이슬점온도(C)
            'hm': split_data[13],   # 상대습도(%)
            'rn': split_data[16],   # 일일 강수량(mm)
            'ss': split_data[33],   # 일조 (hr)
            'si': split_data[34],   # 일사 (MJ/m2)
            'ts': split_data[36],   # 지면온도 (C)
            }
    return None

In [6]:
from tqdm import tqdm

# 날씨 정보를 데이터프레임에 추가하는 함수
def add_weather_data_to_df(df):
    weather_data = []
    for i in tqdm(range(len(df)), desc="Processing rows", unit="row"):
        region = df.iloc[i][1]
        date = f"{df.iloc[i]['year']}{str(df.iloc[i]['month']).zfill(2)}{str(df.iloc[i]['day']).zfill(2)}1300"
        
        # 지역에 맞는 날씨 정보 요청
        lines = download_file(date, str(stn_dic.get(region, '108')))
        
        # 날씨 데이터 추출
        weather = extract_weather_data(lines)
        
        # 데이터가 없으면 서울 지역으로 요청
        if weather is None:
            lines = download_file(date, '108')
            weather = extract_weather_data(lines)
        
        # 결과 추가 (날씨 데이터를 리스트에 추가)
        if weather:
            weather_data.append(weather)
        else:
            weather_data.append({'ws': None, 'pa': None, 'ps': None, 'ta': None, 'td': None, 'hm': None, 
                                 'rn': None, 'ss': None, 'si': None, 'ts': None})
    
    # 각 열을 추가
    df['ws'] = [w['ws'] for w in weather_data]
    df['pa'] = [w['pa'] for w in weather_data]
    df['ps'] = [w['ps'] for w in weather_data]
    df['ta'] = [w['ta'] for w in weather_data]
    df['td'] = [w['td'] for w in weather_data]
    df['hm'] = [w['hm'] for w in weather_data]
    df['rn'] = [w['rn'] for w in weather_data]
    df['ss'] = [w['ss'] for w in weather_data]
    df['si'] = [w['si'] for w in weather_data]
    df['ts'] = [w['ts'] for w in weather_data]
    
    return df

In [8]:
# 날씨 데이터를 데이터프레임에 추가
df_with_weather = add_weather_data_to_df(df)

# 결과 확인
display(df_with_weather)

Processing rows: 100%|██████████| 596/596 [03:39<00:00,  2.71row/s]


,1,2,year,month,day,ws,pa,ps,ta,td,hm,rn,ss,si,ts
0,부산광역시,영도구,2019,1,2,3.9,1021.5,1030.2,4.2,-11.8,30.0,-9.0,1.0,2.12,17.8
1,부산광역시,영도구,2019,1,3,5.1,1023.5,1032.2,5.3,-15.9,20.0,-9.0,1.0,2.14,17.8
2,전라남도,고흥군,2019,1,3,1.4,1024.8,1032.9,4.6,-6.3,45.0,-9.0,1.0,1.74,19.0
3,전라남도,목포시,2019,1,5,4.4,1015.3,1023.3,7.0,-5.7,40.0,-9.0,0.3,0.79,15.4
4,경상남도,거제시,2019,1,10,4.0,1012.9,1021.5,7.5,-13.4,21.0,-9.0,1.0,2.09,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,경상남도,창원시 마산합포구,2023,11,24,4.3,1015.8,1024.3,8.5,-13.2,20.0,-9.0,1.0,2.29,21.6
592,제주특별자치도,제주시,2023,12,7,3.6,1015.8,1018.3,16.9,0.5,33.0,-9.0,1.0,2.11,21.4
593,전라남도,목포시,2023,12,16,10.6,1012.2,1020.4,3.4,-3.6,60.0,0.2,0.5,1.51,8.2
594,전라남도,여수시,2023,12,19,1.9,1018.4,1026.6,4.8,-6.4,44.0,0.3,0.6,1.42,11.7


In [11]:
df_with_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       596 non-null    object
 1   2       596 non-null    object
 2   year    596 non-null    int32 
 3   month   596 non-null    int32 
 4   day     596 non-null    int32 
 5   ws      596 non-null    object
 6   pa      596 non-null    object
 7   ps      596 non-null    object
 8   ta      596 non-null    object
 9   td      596 non-null    object
 10  hm      596 non-null    object
 11  rn      596 non-null    object
 12  ss      596 non-null    object
 13  si      596 non-null    object
 14  ts      596 non-null    object
dtypes: int32(3), object(12)
memory usage: 63.0+ KB


In [12]:
df_with_weather.head()

,1,2,year,month,day,ws,pa,ps,ta,td,hm,rn,ss,si,ts
0,부산광역시,영도구,2019,1,2,3.9,1021.5,1030.2,4.2,-11.8,30.0,-9.0,1.0,2.12,17.8
1,부산광역시,영도구,2019,1,3,5.1,1023.5,1032.2,5.3,-15.9,20.0,-9.0,1.0,2.14,17.8
2,전라남도,고흥군,2019,1,3,1.4,1024.8,1032.9,4.6,-6.3,45.0,-9.0,1.0,1.74,19.0
3,전라남도,목포시,2019,1,5,4.4,1015.3,1023.3,7.0,-5.7,40.0,-9.0,0.3,0.79,15.4
4,경상남도,거제시,2019,1,10,4.0,1012.9,1021.5,7.5,-13.4,21.0,-9.0,1.0,2.09,20.0


In [13]:
# -9.0 값을 0으로 변경
df_with_weather['rn'] = df_with_weather['rn'].replace('-9.0', '0')
# 컬럼을 int로 변환
df_with_weather['rn'] = df_with_weather['rn'].astype(float)
# -9.0을 0으로 바꾸기
df_with_weather['ws'] = df_with_weather['ws'].replace(-9.0, 0)
df_with_weather['hm'] = df_with_weather['hm'].replace(-9.0, 0)

In [14]:
df_with_weather.head()

,1,2,year,month,day,ws,pa,ps,ta,td,hm,rn,ss,si,ts
0,부산광역시,영도구,2019,1,2,3.9,1021.5,1030.2,4.2,-11.8,30.0,0.0,1.0,2.12,17.8
1,부산광역시,영도구,2019,1,3,5.1,1023.5,1032.2,5.3,-15.9,20.0,0.0,1.0,2.14,17.8
2,전라남도,고흥군,2019,1,3,1.4,1024.8,1032.9,4.6,-6.3,45.0,0.0,1.0,1.74,19.0
3,전라남도,목포시,2019,1,5,4.4,1015.3,1023.3,7.0,-5.7,40.0,0.0,0.3,0.79,15.4
4,경상남도,거제시,2019,1,10,4.0,1012.9,1021.5,7.5,-13.4,21.0,0.0,1.0,2.09,20.0


In [16]:
df_with_weather = df_with_weather.drop(1, axis=1).drop(2, axis=1)
# 결과 컬럼 추가 및 화재 발생(1)으로 기록
df_with_weather['result'] = 1
df_with_weather.head()

,year,month,day,ws,pa,ps,ta,td,hm,rn,ss,si,ts,result
0,2019,1,2,3.9,1021.5,1030.2,4.2,-11.8,30.0,0.0,1.0,2.12,17.8,1
1,2019,1,3,5.1,1023.5,1032.2,5.3,-15.9,20.0,0.0,1.0,2.14,17.8,1
2,2019,1,3,1.4,1024.8,1032.9,4.6,-6.3,45.0,0.0,1.0,1.74,19.0,1
3,2019,1,5,4.4,1015.3,1023.3,7.0,-5.7,40.0,0.0,0.3,0.79,15.4,1
4,2019,1,10,4.0,1012.9,1021.5,7.5,-13.4,21.0,0.0,1.0,2.09,20.0,1


In [19]:
df_with_weather.to_excel('선박 화재 발생O.xlsx', sheet_name='WeatherData', index=False)

# 화재 x 날씨 데이터

In [20]:
import pandas as pd
from tqdm import tqdm 

# 빈 데이터프레임 초기화
df = pd.DataFrame(columns=['year', 'month', 'day', 'ws', 'ta', 'hm', 'rn', 'result'])

# 연도, 월, 일 범위 설정
years = range(2019, 2024)
months = range(1, 13)
days = range(1, 32)

# tqdm으로 진행 바 추가
total_days = len(years) * len(months) * len(days)
with tqdm(total=total_days, desc="Processing dates", unit="day") as pbar:
    i = 0
    for y in years:
        for m in months:
            for d in days:
                # 날짜 형식을 yymmdd로 변환
                target_date = f"{str(y)}{str(m).zfill(2)}{str(d).zfill(2)}1300"    
                com_date = f"{str(df_with_weather.iloc[i]['year'])}{str(df_with_weather.iloc[i]['month']).zfill(2)}{str(df_with_weather.iloc[i]['day']).zfill(2)}"
                
                # 만약 화재가 발생한 날짜라면 건너뛴다.
                if target_date == com_date:
                    i += 1
                    continue
                else:
                    # 날씨 데이터 다운로드
                    lines = download_file(target_date, str(159))
                    # display(lines)
                    # 날씨 데이터 추출
                    weather = extract_weather_data(lines)
                    # print(target_date)
                    # print(weather['ws'] + ' ' + weather['ta'] + ' ' + weather['hm'] + ' ' + weather['rn'])

                    # 새로운 데이터를 리스트에 추가
                    if weather:
                        new_data = {
                            'year': y, 'month': m, 'day': d, 
                            'ws': weather['ws'], 'pa': weather['pa'], 
                            'ps': weather['ps'], 'ta': weather['ta'], 
                            'td': weather['td'], 'hm': weather['hm'], 
                            'rn': weather['rn'], 'ss': weather['ss'], 
                            'si': weather['si'], 'ts': weather['ts'], 
                            'result': 0
                        }
                    else:
                        new_data = {
                            'year': y, 'month': m, 'day': d, 
                            'ws': None, 'pa': None, 
                            'ps': None, 'ta': None, 
                            'td': None, 'hm': None, 
                            'rn': None, 'ss': None, 
                            'si': None, 'ts': None,
                            'result': 0
                        }
                    
                    # 데이터프레임에 새 데이터 추가
                    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)

                # 진행 바 업데이트
                pbar.update(1)
# 결과 데이터프레임 출력
print(df.head())


Processing dates: 100%|██████████| 1860/1860 [13:14<00:00,  2.34day/s]

   year month day   ws   ta    hm    rn result      pa      ps     td   ss  \
0  2019     1   1  3.0  4.1  36.0  -9.0      0  1019.9  1028.6   -9.6  1.0   
1  2019     1   2  3.9  4.2  30.0  -9.0      0  1021.5  1030.2  -11.8  1.0   
2  2019     1   3  5.1  5.3  20.0  -9.0      0  1023.5  1032.2  -15.9  1.0   
3  2019     1   4  5.8  6.3  20.0  -9.0      0  1019.7  1028.4  -15.0  0.1   
4  2019     1   5  2.5  8.5  30.0  -9.0      0  1013.7  1022.2   -8.1  0.9   

     si    ts  
0  2.08  17.1  
1  2.12  17.8  
2  2.14  17.8  
3  0.83   9.8  
4  1.46  19.0  


### 2월 30일 같은 없는 날짜의 값을 제거한다.

In [21]:
df = df.dropna()

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1825 entries, 0 to 1859
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    1825 non-null   object
 1   month   1825 non-null   object
 2   day     1825 non-null   object
 3   ws      1825 non-null   object
 4   ta      1825 non-null   object
 5   hm      1825 non-null   object
 6   rn      1825 non-null   object
 7   result  1825 non-null   object
 8   pa      1825 non-null   object
 9   ps      1825 non-null   object
 10  td      1825 non-null   object
 11  ss      1825 non-null   object
 12  si      1825 non-null   object
 13  ts      1825 non-null   object
dtypes: object(14)
memory usage: 213.9+ KB


In [23]:
# -9.0 값을 0으로 변경
df['rn'] = df['rn'].replace('-9.0', '0')
# 컬럼을 int로 변환
df['rn'] = df['rn'].astype(float)
# -9.0을 0으로 바꾸기
df['ws'] = df['ws'].replace(-9.0, 0)
df['hm'] = df['hm'].replace(-9.0, 0)

In [24]:
df.sample(10)

,year,month,day,ws,ta,hm,rn,result,pa,ps,td,ss,si,ts
743,2020,12,31,3.9,-0.7,37.0,0.0,0,1013.8,1022.6,-13.6,1.0,2.17,3.4
1411,2022,10,17,2.1,23.7,37.0,0.0,0,1004.4,1012.3,8.1,1.0,2.58,39.7
115,2019,4,23,0.9,18.2,68.0,0.0,0,1006.6,1014.7,12.1,0.0,0.79,21.0
235,2019,8,19,3.2,29.3,73.0,0.0,0,1001.5,1009.3,23.9,1.0,2.60,37.2
1796,2023,10,30,2.2,22.3,37.0,0.0,0,1012.6,1020.7,6.8,1.0,2.42,45.2
190,2019,7,5,2.4,26.9,62.0,0.0,0,995.8,1003.6,18.9,1.0,3.17,34.6
1600,2023,4,20,6.1,21.9,70.0,0.0,0,999.9,1007.9,16.1,1.0,3.36,36.1
284,2019,10,6,4.5,21.5,51.0,0.0,0,1014.4,1022.5,10.9,1.0,2.24,24.8
4,2019,1,5,2.5,8.5,30.0,0.0,0,1013.7,1022.2,-8.1,0.9,1.46,19.0
744,2021,1,1,3.2,3.6,39.0,0.0,0,1015.8,1024.5,-9.0,1.0,2.11,9.7


In [25]:
df.to_excel('선박 화재 발생x.xlsx', sheet_name='WeatherData', index=False)

# 앞으로 할 일 
화재 발생시 일자가 겹치는 컬럼이 다수 존재 -> 중복 제거    
화재 발생x 데이터에서는 2019~2024모든 일의 컬럼이 존재 -> 화재 발생시 데이터를 삽입하며 화재 발생 x에 존재하던 해당 일자의 데이터를 삭제 후 삽입한다.

In [26]:
import pandas as pd

In [27]:
df = pd.read_excel('선박 화재 발생O.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    596 non-null    int64  
 1   month   596 non-null    int64  
 2   day     596 non-null    int64  
 3   ws      596 non-null    float64
 4   pa      596 non-null    float64
 5   ps      596 non-null    float64
 6   ta      596 non-null    float64
 7   td      596 non-null    float64
 8   hm      596 non-null    float64
 9   rn      596 non-null    float64
 10  ss      596 non-null    float64
 11  si      596 non-null    float64
 12  ts      596 non-null    float64
 13  result  596 non-null    int64  
dtypes: float64(10), int64(4)
memory usage: 65.3 KB


In [28]:
# 중복 확인을 위한 기준이 year, month, day인 경우
def select_row_based_on_conditions(group):
    if group.empty:
        return None  # 빈 그룹 처리

    # 조건 1: hm가 낮고 rn가 낮으며 ta가 높은 행 선택
    selected_rows = group.loc[(group['hm'] == group['hm'].min()) &
                              (group['rn'] == group['rn'].min()) &
                              (group['ta'] == group['ta'].max())]
    
    # 조건 2: 만약 선택된 행들이 여러 개라면, ws가 높은 행 선택
    if len(selected_rows) > 1:
        selected_rows = selected_rows.loc[selected_rows['ws'] == selected_rows['ws'].max()]
    
    # 조건 3: 그래도 여러 행이 남아있다면, 첫 번째 행을 선택
    return selected_rows.iloc[0] if not selected_rows.empty else group.iloc[0]

# year, month, day로 그룹핑 후 조건에 맞는 행만 남김
df_cleaned = df.groupby(['year', 'month', 'day'], group_keys=False).apply(select_row_based_on_conditions)

# 결측값(빈 그룹에서 반환된 None) 제거
df_cleaned = df_cleaned.dropna()

# 결과 확인
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 509 entries, (2019, 1, 2) to (2023, 12, 21)
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    509 non-null    float64
 1   month   509 non-null    float64
 2   day     509 non-null    float64
 3   ws      509 non-null    float64
 4   pa      509 non-null    float64
 5   ps      509 non-null    float64
 6   ta      509 non-null    float64
 7   td      509 non-null    float64
 8   hm      509 non-null    float64
 9   rn      509 non-null    float64
 10  ss      509 non-null    float64
 11  si      509 non-null    float64
 12  ts      509 non-null    float64
 13  result  509 non-null    float64
dtypes: float64(14)
memory usage: 57.7 KB


In [29]:
# 중복 처리된 데이터프레임을 저장
df_cleaned.to_excel('중복 제거 선박 화재O.xlsx', index=False)

# 데이터 프레임 합치기

In [30]:
import pandas as pd

In [31]:
df_o = pd.read_excel('중복 제거 선박 화재O.xlsx')
df_x = pd.read_excel('선박 화재 발생x.xlsx')

In [32]:
display(len(df_o))
display(len(df_x))

509

1825

In [33]:
# 두 DataFrame을 세로로 병합합니다.
df_merged = pd.concat([df_x, df_o], ignore_index=True)
display(df_merged.info())
df_merged.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2334 entries, 0 to 2333
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    2334 non-null   int64  
 1   month   2334 non-null   int64  
 2   day     2334 non-null   int64  
 3   ws      2334 non-null   float64
 4   ta      2334 non-null   float64
 5   hm      2334 non-null   float64
 6   rn      2334 non-null   float64
 7   result  2334 non-null   int64  
 8   pa      2334 non-null   float64
 9   ps      2334 non-null   float64
 10  td      2334 non-null   float64
 11  ss      2334 non-null   float64
 12  si      2334 non-null   float64
 13  ts      2334 non-null   float64
dtypes: float64(10), int64(4)
memory usage: 255.4 KB


None

,year,month,day,ws,ta,hm,rn,result,pa,ps,td,ss,si,ts
0,2019,1,1,3.0,4.1,36.0,0.0,0,1019.9,1028.6,-9.6,1.0,2.08,17.1
1,2019,1,2,3.9,4.2,30.0,0.0,0,1021.5,1030.2,-11.8,1.0,2.12,17.8
2,2019,1,3,5.1,5.3,20.0,0.0,0,1023.5,1032.2,-15.9,1.0,2.14,17.8
3,2019,1,4,5.8,6.3,20.0,0.0,0,1019.7,1028.4,-15.0,0.1,0.83,9.8
4,2019,1,5,2.5,8.5,30.0,0.0,0,1013.7,1022.2,-8.1,0.9,1.46,19.0


In [34]:
# year, month, day가 중복되는 행 중 result가 0인 행 삭제
df_merged = df_merged[~((df_merged.duplicated(subset=['year', 'month', 'day'], keep=False)) & (df_merged['result'] == 0))]
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1825 entries, 0 to 2333
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    1825 non-null   int64  
 1   month   1825 non-null   int64  
 2   day     1825 non-null   int64  
 3   ws      1825 non-null   float64
 4   ta      1825 non-null   float64
 5   hm      1825 non-null   float64
 6   rn      1825 non-null   float64
 7   result  1825 non-null   int64  
 8   pa      1825 non-null   float64
 9   ps      1825 non-null   float64
 10  td      1825 non-null   float64
 11  ss      1825 non-null   float64
 12  si      1825 non-null   float64
 13  ts      1825 non-null   float64
dtypes: float64(10), int64(4)
memory usage: 213.9 KB


In [35]:
# year, month, day 기준으로 정렬
df_sorted = df_merged.sort_values(by=['year', 'month', 'day'])
df_sorted.head()

,year,month,day,ws,ta,hm,rn,result,pa,ps,td,ss,si,ts
0,2019,1,1,3.0,4.1,36.0,0.0,0,1019.9,1028.6,-9.6,1.0,2.08,17.1
1825,2019,1,2,3.9,4.2,30.0,0.0,1,1021.5,1030.2,-11.8,1.0,2.12,17.8
1826,2019,1,3,5.1,5.3,20.0,0.0,1,1023.5,1032.2,-15.9,1.0,2.14,17.8
3,2019,1,4,5.8,6.3,20.0,0.0,0,1019.7,1028.4,-15.0,0.1,0.83,9.8
1827,2019,1,5,4.4,7.0,40.0,0.0,1,1015.3,1023.3,-5.7,0.3,0.79,15.4


In [36]:
# 데이터프레임을 저장
df_sorted.to_excel('최종본.xlsx', index=False)

# 이상치 제거

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('최종본.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    1825 non-null   int64  
 1   month   1825 non-null   int64  
 2   day     1825 non-null   int64  
 3   ws      1825 non-null   float64
 4   ta      1825 non-null   float64
 5   hm      1825 non-null   int64  
 6   rn      1825 non-null   float64
 7   result  1825 non-null   int64  
dtypes: float64(3), int64(5)
memory usage: 114.2 KB


In [9]:
# -9.0을 0으로 바꾸기
df['ws'] = df['ws'].replace(-9.0, 0)
df['hm'] = df['hm'].replace(-9.0, 0)

In [10]:
# 데이터프레임을 저장
df.to_excel('최종본.xlsx', index=False)